In [2]:
import os
os.chdir("/workspace")
from Korpora import Korpora
from tqdm import tqdm
from kiwipiepy import Kiwi
# Korpora.fetch("kowikitext", root_dir="/workspace/Korpora")
# Korpora.fetch("namuwikitext", root_dir="/workspace/Korpora")
from tools.tokenizer_tool import nori_NNP

In [3]:
nori_NNP("ㅋㅋㅋ 이런 것도 가능 하냐 ㅋㅋ ?")

['ㅋㅋㅋ', '이런', '것', '도', '가능', '하', '냐', 'ㅋㅋ']

In [4]:
kiwi_tokenizer = Kiwi(num_workers=0, load_default_dict=True, model_type="sbg")
kiwi_tokenizer.tokenize("")

def kiwi_run(text:str, saisiot:bool=True) -> list[str]:
    result = kiwi_tokenizer.tokenize(text, saisiot=saisiot, split_complex=True)
    token_forms = [token.form for token in result]
    return token_forms

def kiwi_run_for_bbpe(text:str, saisiot:bool=True) -> str:
    token_forms = kiwi_run(text, saisiot=saisiot)
    tokenized_item = " ".join(token_forms)
    return tokenized_item

def nori_for_bbpe(text:str):
    token_forms = nori_NNP(text)
    tokenized_item = " ".join(token_forms)
    return tokenized_item
    

In [19]:
kiwi_run_for_bbpe("ㅋㅋㅋ 이런 것도 가능 하냐 ㅋㅋ...")

'ㅋㅋㅋ 이런 것 도 가능 하 냐 ㅋㅋ ...'

In [ ]:
from Korpora import Korpora
corpus = Korpora.load("namuwikitext", root_dir="/workspace/Korpora")

In [17]:
kiwi_run_for_bbpe(corpus.train[3].text)

"羽衣小町 《 아이돌 마스터 신데렐라 걸즈 》 에 등장 하 는 코바야카 와 사 에 , 시 오미 슈코 로 이루 어 지 ᆫ 2 인 유닛 . 통칭 ' 슈사에 ' . 둘 다 교토 가 고향 이 고 일본 전통 컨셉 아이돌 이 라는 공통점 이 있 다 . 또한 이 둘 은 아이돌 을 하 는 데 있 어서 집안 과 의 갈등 을 포함 하 고 있 다는 공통점 도 있 다 . 사 에 는 엄격 하 ᆫ 부모 님 의 반대 를 무릅쓰 고 상경 , 슈코 는 화과자 집 가업 을 잇 기 싫 어서 가출 ( ... ) 모바마스 에서 둘 의 상호 아이돌 토크 가 있 고 ' 하고로모코마치 ' 이 라는 유닛 명 을 달 고 나오 어 여러 이벤트 에서 출연 하 고 있 다 . 유닛 명 은 ' 날개옷 미녀 ' 이 라는 뜻 이 다 . MAGIC HOUR SP 9 화 에서 슈코 와 사 에 단 둘 이서 만 나오 어서 직접 유닛 명 을 언급 하 었 다 . # 신데메이션 25 화 에서 사에 , 슈코 , 아즈키 , 카코 가 함께 행사 를 꾸리 는 장면 이 스치 듯 지나 어 가 었 다 . 신데렐라 걸즈 극장 애니 5 화 에서 사 에 , 슈코 가 카오루 를 데리 고 꽃 구경 을 가 었 다 . 2016 년 10 월 15 일 에 열리 ᆫ 신데마스 4 thLIVE SSA 공연 에서 사에 와 슈코 의 성우 가 青 の 一番星 를 듀엣 으로 부르 는 무대 가 있 었 다 . 데레스테 에서 사에 의 개인 커뮤 에 슈코 가 등장 하 거나 , 사에 의 SSR 에 슈코가 같이 그리 어 지 는 등 친분 관계 가 지속 적 으로 묘사 되 고 있 다 . 2018 년 1 월 12 일 에 데레스테 를 통하 어 유닛 곡 美 に 入 り 彩 を 穿 つ 가 나오 게 되 었 다 . 특이 하 게 도 데레스테 에서 쿨 속성 의 다크 일루미 네이트 처럼 솔로 곡 이 ᆫ 花簪 HANAKANZASHI 와 青 の 一番星 는 물론 , 듀엣 곡 이 ᆫ 美 に 入 り 彩 を 穿 つ 까지 MASTER 레벨 이 모두 27 로 통일 되 어 있 고 , 셋 다 난이도 가 27 상위 권 이 라

In [ ]:
with open("/workspace/data/corpus_data/korean_tokenized_corpus.txt", "w", encoding="utf-8") as f:
    for number, text in enumerate(corpus.train):
        text_data = text.text
        try:
            tokenized_text = kiwi_run_for_bbpe(text_data)
            f.write(tokenized_text + "\n")
        except Exception as e:
            print(f"Error Occur = {e}")
        
        if (number+1) % 100 == 0:
            print(f"{(number+1)} / {len(corpus.train)}")

In [ ]:
from tokenizers import ByteLevelBPETokenizer

new_tokenizer = ByteLevelBPETokenizer()

new_tokenizer.train(
    files="/workspace/data/corpus_data/korean_tokenized_corpus.txt", 
    vocab_size=55000, 
    min_frequency=6, 
    special_tokens=["[PAD]", "[UNK]", "[CLS]", "[SEP]", "[MASK]"],
    show_progress=True,
    )

new_tokenizer.save_model("/workspace/result/tokenizer_beta")

In [36]:
new_tokenizer.save("/workspace/result/tokenizer_beta/tokenizer.json")


In [17]:
import json
import copy

# 1. 두 tokenizer.json 파일 경로
tokenizer_path_1 = "/workspace/ModernBERT-base/tokenizer.json"
tokenizer_path_2 = "/workspace/result/tokenizer_beta/tokenizer.json"

# 2. tokenizer.json 파일 읽기
with open(tokenizer_path_1, "r", encoding="utf-8") as f1:
    tokenizer_1 = json.load(f1)

with open(tokenizer_path_2, "r", encoding="utf-8") as f2:
    tokenizer_2 = json.load(f2)

# 3. 1st tokenizer의 vocab을 그대로 유지하고, 2nd tokenizer의 vocab 추가
vocab_1 = tokenizer_1['model']["vocab"]
vocab_2 = tokenizer_2['model']["vocab"]

added_tokens = tokenizer_1['added_tokens']
added_tokens_list = [token['content'] for token in added_tokens]

special_tokens = {
    "[CLS]": 50281,
    "[MASK]": 50284,
    "[PAD]": 50283,
    "[SEP]": 50282,
    "[UNK]": 50280
}

# vocab 병합 (중복된 토큰은 제외하고 추가)
new_idx_number = 0
merged_vocab = vocab_1.copy()
merged_vocab.update(special_tokens)
start_size = len(merged_vocab)

for token, idx in vocab_2.items():
    if token not in merged_vocab and token not in added_tokens_list:
        merged_vocab[token] = new_idx_number + start_size
        new_idx_number += 1

# 4. 1st tokenizer의 merges와 2nd tokenizer의 merges 병합 (중복된 항목 제외)
merges_1 = tokenizer_1['model']["merges"]
merges_2 = tokenizer_2['model']['merges']
merges_2_strings = [" ".join(merge) for merge in merges_2]

merged_merges = copy.deepcopy(merges_1)
for item in merges_2_strings:
    if item not in merges_1:
        merged_merges.append(item)
    
new_tokenizer = copy.deepcopy(tokenizer_1)
new_tokenizer['model']['vocab'] = merged_vocab
new_tokenizer['model']['merges'] = merged_merges

merged_tokenizer_path = "/workspace/result/tok_ber_2/tokenizer.json"
with open(merged_tokenizer_path, "w", encoding="utf-8") as f:
    json.dump(new_tokenizer, f, ensure_ascii=False, indent=4)

In [18]:
from transformers import AutoTokenizer, AutoModelForMaskedLM

tokenizer = AutoTokenizer.from_pretrained("/workspace/result/new_tokenizer")
tokenizer.save_pretrained("/workspace/result/new_tokenizer_2")

('/workspace/result/new_tokenizer_2/tokenizer_config.json',
 '/workspace/result/new_tokenizer_2/special_tokens_map.json',
 '/workspace/result/new_tokenizer_2/tokenizer.json')

In [7]:
import json
from transformers import AutoTokenizer
from collections import OrderedDict

# tokenizer.json 파일 로드
tokenizer_path = "/workspace/result/new_tokenizer/tokenizer.json"

with open(tokenizer_path, 'r', encoding='utf-8') as f:
    tokenizer_json = json.load(f)

# 'vocab' 부분만 가져오기 (vocab은 어휘를 포함하는 부분)
vocab = tokenizer_json['model']['vocab']

# 불연속적인 인덱스를 제거하고 새롭게 연속적인 인덱스를 할당
new_vocab = OrderedDict()
new_idx = 0  # 새로운 인덱스 번호

# 기존 어휘의 순서를 유지하며, 인덱스를 새롭게 부여
for token, idx in vocab.items():
    new_vocab[token] = new_idx
    new_idx += 1

# 새로 정리된 vocab을 'tokenizer.json'에 반영
tokenizer_json['model']['vocab'] = new_vocab

# 새로운 tokenizer.json 파일로 저장
new_tokenizer_path = "/workspace/result/new_tokenizer"
with open(f"{new_tokenizer_path}/tokenizer_2.json", 'w', encoding='utf-8') as f:
    json.dump(tokenizer_json, f, ensure_ascii=False, indent=2)

# 새로 만든 토크나이저 로드
tokenizer = AutoTokenizer.from_pretrained(new_tokenizer_path)

# 새로운 토크나이저 저장
tokenizer.save_pretrained(new_tokenizer_path)

print("Tokenizer with continuous index saved at:", new_tokenizer_path)


The OrderedVocab you are attempting to save contains holes for indices [50285, 50286, 50287, 50288, 50289, 50290, 50291, 50292, 50293, 50294, 50295, 50296, 50297, 50298, 50299, 50300, 50301, 50302, 50303, 50304, 50305, 50306, 50307, 50308, 50309, 50310, 50311, 50312, 50313, 50314, 50315, 50316, 50317, 50318, 50319, 50320, 50321, 50322, 50323, 50324, 50325, 50326, 50327, 50328, 50329, 50330, 50331, 50332, 50333, 50334, 50335, 50336, 50337, 50338, 50339, 50340, 50341, 50342, 50343, 50344, 50345, 50346, 50347, 50348, 50349, 50350, 50351, 50352, 50353, 50354, 50355, 50356, 50357, 50358, 50359, 50360, 50361, 50362, 50363, 50364, 50365, 50366, 50367, 50368, 50369, 50370, 50371, 50372, 50373, 50374, 50375, 50376, 50377, 50378, 50379, 50380, 50381, 50382, 50383, 50384, 50385, 50386, 50387, 50388, 50389, 50390, 50391, 50392, 50393, 50394, 50395, 50396, 50397, 50398, 50399, 50400, 50401, 50402, 50403, 50404, 50405, 50406, 50407, 50408, 50411, 50412, 50413, 50414, 50415, 50416, 50417, 50418, 5041

In [85]:
len(tokenizer.get_vocab())

98239

In [ ]:
tokenizer.tokenize("BBPE 토크나이저는 토큰을 Byte 형식으로 나눕니다.")

In [ ]:
tokenizer.encode("하하하 이런것도 처리할 수 있겠느냐 ㅋㅋ")

In [2]:
corpus = Korpora.load("kowikitext", root_dir="/workspace/Korpora")


    Korpora 는 다른 분들이 연구 목적으로 공유해주신 말뭉치들을
    손쉽게 다운로드, 사용할 수 있는 기능만을 제공합니다.

    말뭉치들을 공유해 주신 분들에게 감사드리며, 각 말뭉치 별 설명과 라이센스를 공유 드립니다.
    해당 말뭉치에 대해 자세히 알고 싶으신 분은 아래의 description 을 참고,
    해당 말뭉치를 연구/상용의 목적으로 이용하실 때에는 아래의 라이센스를 참고해 주시기 바랍니다.

    # Description
    Author : Hyunjoong Kim lovit@github
    Repository : https://github.com/lovit/kowikitext
    References :

    한국어 위키피디아의 덤프 데이터를 바탕을 제작한 wikitext 형식의 텍스트 파일입니다.
    학습 및 평가를 위하여 위키페이지 별로 train (99%), dev (0.5%), test (0.5%) 로 나뉘어져있습니다.


    # License
    CC-BY-SA 3.0 which kowiki dump dataset is licensed

[Korpora] Corpus `kowikitext` is already installed at /workspace/Korpora/kowikitext/kowikitext_20200920.train.zip
[Korpora] Corpus `kowikitext` is already installed at /workspace/Korpora/kowikitext/kowikitext_20200920.train
[Korpora] Corpus `kowikitext` is already installed at /workspace/Korpora/kowikitext/kowikitext_20200920.test.zip
[Korpora] Corpus `kowikitext` is already installed at /workspace/Korpora/kowikitext/kowik

In [ ]:
with open("/workspace/data/corpus_data/korean_train_corpus.txt", "w", encoding="utf-8") as f:
    for number, text in enumerate(corpus.train):
        text_data = text.text
        try:
            f.write(text_data + "\n")
        except Exception as e:
            print(f"Error Occur = {e}")
        
        if (number+1) % 100 == 0:
            print(f"{(number+1)} / {len(corpus.train)}")